In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-11-30"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
21773,2024-11-30,Brasil Nbb,15:00,Flamengo,Minas,1.41,2.62,152.5,1.83,1.83,-6.5,1.98,1.70,SAID3h1s,0.709220,0.381679,0.546448,0.546448,0.090899,0.4,0.4,NaN,NaN,98.372,8.900004,0.090473,108.374,16.095853,0.148521,85.178,135.570,92.82,125.49,0.0,0.0,0.0,0.0,0.424615,0.000000,0.107603,-0.83,-0.166,-2.469880,0.919298,0.9,-0.019298,0.65,0.130,12.461538,0.704640,0.8,0.095360
21774,2024-11-30,Brasil Nbb,18:00,Bauru,Unifacisa,1.59,2.16,159.5,1.84,1.82,-4.5,2.02,1.67,zyML1WVg,0.628931,0.462963,0.543478,0.549451,0.091894,0.4,0.4,NaN,NaN,143.304,64.040193,0.446883,177.118,79.532785,0.449038,141.678,165.630,105.60,269.80,0.0,0.0,0.0,0.0,0.214960,0.007728,0.134139,1.91,0.382,1.544503,0.649768,0.8,0.150232,-3.34,-0.668,-1.736527,0.485552,0.2,-0.285552
21775,2024-11-30,Espanha Acb,14:00,Breogan,Granada,1.54,2.50,155.5,1.88,1.92,-5.5,1.99,1.69,OhlKYBBG,0.649351,0.400000,0.531915,0.520833,0.049351,0.6,0.2,NaN,NaN,231.288,109.644254,0.474059,255.742,115.669469,0.452290,217.494,333.962,396.55,451.50,1.0,0.0,0.0,0.0,0.336051,0.014886,0.115289,1.70,0.340,1.588235,0.465083,0.3,-0.165083,0.49,0.098,15.306122,0.281649,0.4,0.118351
21776,2024-11-30,Espanha Acb,14:00,Unicaja,Tenerife,1.31,3.51,167.5,1.87,1.87,-8.5,1.98,1.77,UgeBzYt4,0.763359,0.284900,0.534759,0.534759,0.048259,0.6,0.6,NaN,NaN,115.870,31.620422,0.272896,143.570,48.987204,0.341208,135.182,131.220,170.98,127.89,0.0,0.0,0.0,0.0,0.645492,0.000000,0.079196,0.20,0.040,7.750000,0.000000,0.0,0.000000,1.02,0.204,12.303922,0.567592,0.8,0.232408
21777,2024-11-30,Espanha Acb,16:45,Basquet Girona,MoraBanc Andorra,2.11,1.73,167.5,1.88,1.92,1.5,1.97,1.83,xfNlVoO7,0.473934,0.578035,0.531915,0.520833,0.051968,0.2,0.6,NaN,NaN,255.300,163.291580,0.639607,176.106,106.859977,0.606794,152.770,272.848,243.00,119.00,0.0,0.0,0.0,0.0,0.139948,0.014886,0.052103,-1.09,-0.218,-5.091743,0.557075,0.4,-0.157075,-5.00,-1.000,-0.730000,0.347713,0.0,-0.347713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22079,2024-11-30,Noruega Blno Feminina,11:00,Bergen F,Bærum F,5.17,1.11,105.5,1.83,1.83,12.5,1.83,1.83,Sb63Yapn,0.193424,0.900901,0.546448,0.546448,0.094324,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,94.57,0.0,0.0,0.0,0.0,0.914285,0.000000,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
22080,2024-11-30,Reino Unido Slb,15:00,Surrey 89ers,Bristol Flyers,1.46,2.47,167.5,1.86,1.80,-7.5,2.01,1.68,YXdUxZVN,0.684932,0.404858,0.537634,0.555556,0.089790,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,228.16,162.00,0.0,0.0,0.0,0.0,0.363449,0.023184,0.126474,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
22081,2024-11-30,República Tcheca Zbl Feminina,13:00,Brandys nad Labem F,Ostrava F,1.59,2.26,139.5,1.88,1.79,-3.5,1.91,1.79,GIcgAtKb,0.628931,0.442478,0.531915,0.558659,0.071409,0.8,0.0,NaN,NaN,247.432,272.352894,1.100718,342.242,437.254343,1.277617,229.624,352.090,132.00,69.69,0.0,0.0,0.0,0.0,0.246110,0.034681,0.045866,-1.26,-0.252,-2.341270,0.000000,0.0,0.000000,4.91,0.982,1.283096,0.000000,0.0,0.000000
22082,2024-11-30,Rússia Premier League Feminina,08:30,Nadezhda F,Neftyanik Omsk F,1.02,10.50,134.5,1.85,1.85,-18.5,2.05,1.59,WCtYDwnN,0.980392,0.095238,0.540541,0.540541,0.075630,0.6,0.6,NaN,NaN,248.982,249.540307,1.

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Los Angeles Lakers,Oklahoma City Thunder,229.5,1.81,0.8806,Over
1,00:00,Eua Nba,Portland Trail Blazers,Sacramento Kings,224.5,1.85,0.9608,Over
2,20:00,Eua Nba,Charlotte Hornets,Atlanta Hawks,226.5,1.81,0.9841,Over
3,21:00,Eua Nba,Detroit Pistons,Philadelphia 76ers,216.5,1.85,0.9359,Over
4,22:00,Eua Nba,Milwaukee Bucks,Washington Wizards,229.5,1.82,0.9793,Over
5,23:00,Eua Nba,Phoenix Suns,Golden State Warriors,231.5,1.84,0.8447,Over
6,23:30,Eua Nba,Utah Jazz,Dallas Mavericks,232.5,1.85,0.9491,Over
